JUEGO DE 4 EN RAYA AGENTE DE APRENDIZAJE POR REFUERZO UTILIZANDO Q-LEARNING



Estas se usan para manipulación de datos (NumPy), selección aleatoria y guardado/carga de objetos (pickle)

In [7]:
#IMPORTACION DE LIBRERIAS
import numpy as np
import random
import pickle

Define un tablero de 6 filas y 7 columnas, con la condición de victoria de conectar 4 fichas.

In [8]:
#CONSTANTES
ROWS = 6
COLS = 7
WIN_LENGTH = 4

In [9]:
#Inicializa el tablero como una matriz de ceros (vacía).

#El jugador actual es 1 (puede alternar con -1 más adelante).
class Connect4:
    def __init__(self):
        self.board = np.zeros((ROWS, COLS), dtype=int)
        self.current_player = 1
#Reinicia el tablero y el turno.

#Devuelve el estado actual del tablero como una tupla.
    def reset(self):
        self.board[:] = 0
        self.current_player = 1
        return self.get_state()
#Convierte el tablero 2D en una tupla 1D. Esto es útil para aprendizaje
#automático (por ejemplo, como entrada para modelos o Q-tables).
    def get_state(self):
        return tuple(self.board.flatten())
#Devuelve una lista de columnas donde todavía se pueden colocar fichas
#(es decir, donde la fila superior está vacía).
    def available_actions(self):
        return [c for c in range(COLS) if self.board[0][c] == 0]
#Este método:

#Verifica si la acción es válida.

#Coloca la ficha del jugador actual en la columna elegida.

#Revisa si el jugador ganó.

#Otorga una recompensa:

#1 si gana,

#0.5 si empata,

#-10 si hace un movimiento inválido,

#0 si el juego continúa.

#Alterna el turno si no ha terminado.

#Devuelve el nuevo estado, la recompensa y si el juego ha terminado.
    def step(self, action):
        if action not in self.available_actions():
            return self.get_state(), -10, True  # Movimiento inválido

        row = self.get_next_open_row(action)
        self.board[row][action] = self.current_player
        done = self.check_winner(self.current_player)

        if done:
            reward = 1
        elif len(self.available_actions()) == 0:
            reward = 0.5  # empate
            done = True
        else:
            reward = 0
            self.current_player *= -1

        return self.get_state(), reward, done
#Encuentra la fila más baja disponible en una columna para colocar una ficha.
    def get_next_open_row(self, col):
        for r in range(ROWS - 1, -1, -1):
            if self.board[r][col] == 0:
                return r
#Comprueba si hay 4 fichas consecutivas del jugador actual en:

#Dirección horizontal

#Dirección vertical

#Diagonal descendente ↘

#Diagonal ascendente ↗

#Si encuentra alguna, devuelve True (hay ganador).
    def check_winner(self, player):
        for c in range(COLS - 3):
            for r in range(ROWS):
                if all(self.board[r, c + i] == player for i in range(WIN_LENGTH)):
                    return True
        for c in range(COLS):
            for r in range(ROWS - 3):
                if all(self.board[r + i, c] == player for i in range(WIN_LENGTH)):
                    return True
        for c in range(COLS - 3):
            for r in range(ROWS - 3):
                if all(self.board[r + i, c + i] == player for i in range(WIN_LENGTH)):
                    return True
        for c in range(COLS - 3):
            for r in range(3, ROWS):
                if all(self.board[r - i, c + i] == player for i in range(WIN_LENGTH)):
                    return True
        return False

In [10]:
class QLearningAgent:
  #INIALIZAMOS
  #cómo debe lucir y qué datos debe tener un objeto de tu clase justo después de ser creado.
    def __init__(self, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.q_table = {}  # {(state, action): value}
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
#te permite consultar cuánto "valor" o "recompensa esperada"
#ha aprendido el agente que se obtiene al realizar una acción específica
#cuando se encuentra en un estado determinado. Si esa combinación estado-acción
#es nueva para el agente, asume un valor inicial de 0.0. Esto es crucial para que
#el agente pueda tomar decisiones basándose en lo que ha aprendido.
    def get_q(self, state, action):
        return self.q_table.get((state, action), 0.0)
#Decide, basándose en el parámetro epsilon, si tomar una acción completamente aleatoria
#para descubrir nuevas posibilidades (exploración) o si tomar la acción que, según su
#tabla Q, se espera que dé la mayor recompensa (explotación).
#Este equilibrio es fundamental para que el agente aprenda de manera efectiva.
    def choose_action(self, state, available_actions):
        if random.random() < self.epsilon:
            return random.choice(available_actions)
        qs = [self.get_q(state, a) for a in available_actions]
        max_q = max(qs)
        max_actions = [a for a, q in zip(available_actions, qs) if q == max_q]
        return random.choice(max_actions)
#Utiliza para refinar la estimación del valor de la acción tomada en el estado anterior.
#Al ajustar gradualmente los valores en la tabla Q basándose en estas experiencias,
#el agente aprende cuáles son las acciones más valiosas en cada estado para maximizar
#la recompensa a largo plazo.
    def update(self, state, action, reward, next_state, next_actions):
        max_q_next = max([self.get_q(next_state, a) for a in next_actions], default=0.0)
        old_q = self.get_q(state, action)
        self.q_table[(state, action)] = old_q + self.alpha * (reward + self.gamma * max_q_next - old_q)

In [11]:
#ENTRENAMIENTO
#es donde el agente de Q-Learning aprende a jugar Connect 4 a través de la interacción
#con el entorno del juego. Implementa el bucle principal de aprendizaje por refuerzo.
env = Connect4()
agent = QLearningAgent()
#Definición del número de episodios
EPISODES = 10000
#Bucle de entrenamiento
for episode in range(EPISODES):
    state = env.reset()
    done = False
#Bucle de juego dentro de cada episodio
    while not done:
        actions = env.available_actions()
        action = agent.choose_action(state, actions)

        next_state, reward, done = env.step(action)
#Turno del oponente (Jugador -1, aquí un oponente aleatorio)
        if not done:
            # El oponente hace una jugada aleatoria
            opp_actions = env.available_actions()
            if opp_actions:
                opp_action = random.choice(opp_actions)
                next_state, reward_opp, done = env.step(opp_action)
                if done:
                    reward = -1 if reward_opp == 1 else reward
#Actualización del agente
        next_actions = env.available_actions()
        agent.update(state, action, reward, next_state, next_actions)
        state = next_state
#Impresión de progreso
    if (episode + 1) % 1000 == 0:
        print(f"Episode {episode + 1}")

# Guardar la tabla Q
with open("q_table_connect4.pkl", "wb") as f:
    pickle.dump(agent.q_table, f)

print("Entrenamiento completado.")

Episode 1000
Episode 2000
Episode 3000
Episode 4000
Episode 5000
Episode 6000
Episode 7000
Episode 8000
Episode 9000
Episode 10000
Entrenamiento completado.


In [12]:
import numpy as np
import pickle

ROWS = 6
COLS = 7
#Re-define la clase Connect4: Vuelve a definir la clase Connect4. Aunque ya estaba definida
#antes, se incluye aquí nuevamente para asegurar que el código de juego tenga acceso a la
#definición del entorno, incluso si se ejecuta esta celda de forma independiente.
class Connect4:
    def __init__(self):
        self.board = np.zeros((ROWS, COLS), dtype=int)
        self.current_player = 1

    def reset(self):
        self.board[:] = 0
        self.current_player = 1
        return self.get_state()

    def get_state(self):
        return tuple(self.board.flatten())

    def available_actions(self):
        return [c for c in range(COLS) if self.board[0][c] == 0]

    def step(self, action):
        if action not in self.available_actions():
            return self.get_state(), -10, True  # Movimiento inválido

        row = self.get_next_open_row(action)
        self.board[row][action] = self.current_player
        done = self.check_winner(self.current_player)

        if done:
            reward = 1
        elif len(self.available_actions()) == 0:
            reward = 0.5
            done = True
        else:
            reward = 0
            self.current_player *= -1

        return self.get_state(), reward, done

    def get_next_open_row(self, col):
        for r in range(ROWS - 1, -1, -1):
            if self.board[r][col] == 0:
                return r

    def check_winner(self, player):
        for c in range(COLS - 3):
            for r in range(ROWS):
                if all(self.board[r, c + i] == player for i in range(4)):
                    return True
        for c in range(COLS):
            for r in range(ROWS - 3):
                if all(self.board[r + i, c] == player for i in range(4)):
                    return True
        for c in range(COLS - 3):
            for r in range(ROWS - 3):
                if all(self.board[r + i, c + i] == player for i in range(4)):
                    return True
        for c in range(COLS - 3):
            for r in range(3, ROWS):
                if all(self.board[r - i, c + i] == player for i in range(4)):
                    return True
        return False

    def print_board(self):
        display = self.board.copy()
        display[display == -1] = 2  # Para mostrar el jugador 2 como "2"
        print(np.flip(display, 0))  # Mostrar con el fondo abajo

# Cargar tabla Q entrada
with open("q_table_connect4.pkl", "rb") as f:
    q_table = pickle.load(f)

def get_q(q_table, state, action):
    return q_table.get((state, action), 0.0)

def agent_choose_action(state, available_actions):
    qs = [get_q(q_table, state, a) for a in available_actions]
    max_q = max(qs)
    best_actions = [a for a, q in zip(available_actions, qs) if q == max_q]
    return np.random.choice(best_actions)

# Jugar contra el agente
env = Connect4()

print("¡Bienvenido a 4 en Raya contra la IA!\nTú eres el jugador 2 (fichas con número 2)")
print("Para jugar, ingresa un número de columna del 0 al 6.\n")
#Bucle principal de juego
while True:
    state = env.reset()
    done = False
    human_player = -1

    while not done:
        env.print_board()

        if env.current_player == human_player:
            try:
                col = int(input("Tu turno (columna 0-6): "))
                if col not in env.available_actions():
                    print("Columna inválida o llena. Intenta otra.")
                    continue
            except ValueError:
                print("Entrada inválida.")
                continue
        else:
            col = agent_choose_action(state, env.available_actions())
            print(f"IA juega columna: {col}")

        next_state, reward, done = env.step(col)
        state = next_state

        if done:
            env.print_board()
            if reward == 1 and env.current_player == human_player:
                print("¡Ganaste!")
            elif reward == 1:
                print("La IA ganó 😞")
            elif reward == 0.5:
                print("¡Empate!")
            elif reward == -10:
                print("Movimiento inválido. Perdiste.")
            break

    again = input("¿Jugar de nuevo? (s/n): ").strip().lower()
    if again != 's':
        break
#Este bloque te permite interactuar directamente con el agente de Q-Learning que entrenaste,
# para ver qué tan bien juega contra un oponente humano

¡Bienvenido a 4 en Raya contra la IA!
Tú eres el jugador 2 (fichas con número 2)
Para jugar, ingresa un número de columna del 0 al 6.

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
IA juega columna: 4
[[0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
Tu turno (columna 0-6): 2
[[0 0 2 0 1 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
IA juega columna: 3
[[0 0 2 1 1 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
Tu turno (columna 0-6): 3
[[0 0 2 1 1 0 0]
 [0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
IA juega columna: 5
[[0 0 2 1 1 1 0]
 [0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
Tu turno (columna 0-6): 4
[[0 0 2 1 1 1 0]
 [0 0 0 2 2 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
IA jue

CONCLUSION  
entrenamiento e interacción con un agente de aprendizaje por refuerzo para jugar al juego Connect 4  
En esencia, estamos haciendo que un programa de computadora aprenda a jugar un juego complejo (Connect 4) simplemente jugando muchas veces y ajustando su estrategia basándose en si sus acciones resultaron en una victoria, una derrota o si le acercaron a un buen estado, y luego te permitimos desafiar a este programa que ha "aprendido".  
-- Definición del Entorno (Connect4 class): Creamos las reglas del juego Connect 4. Esto incluye cómo es el tablero, cómo se hacen los movimientos, cómo se detecta un ganador o un empate, y cómo se representa el estado del juego. Esta clase actúa como el "mundo" con el que el agente interactuará.  
-- Creación y Entrenamiento del Agente (QLearningAgent class y el bucle de entrenamiento):
Diseñamos un agente que utiliza el algoritmo Q-Learning. Este agente tiene una "tabla Q" donde almacenará su conocimiento sobre qué tan buenas son las acciones en diferentes estados del juego.
Sometemos al agente a un proceso de entrenamiento masivo (muchos episodios). En cada episodio, el agente juega una partida, toma decisiones (a veces explorando aleatoriamente, a veces explotando lo aprendido), recibe recompensas del entorno por sus acciones, y utiliza esas recompensas para actualizar y mejorar los valores en su tabla Q. Durante el entrenamiento, el agente aprende a asociar estados del tablero con las acciones que probablemente lo lleven a ganar (o evitar perder).
Guardamos el conocimiento aprendido por el agente (su tabla Q) en un archivo para poder usarlo más tarde.  
Interacción Humano-Agente (el último bloque de código):

Cargamos la tabla Q que el agente entrenado generó.
Creamos una interfaz donde un jugador humano puede jugar partidas de Connect 4 directamente contra el agente entrenado.
El agente utiliza la tabla Q cargada para tomar sus decisiones de juego (siempre eligiendo la mejor opción conocida, sin exploración), mientras que el humano ingresa sus movimientos.  
Interacción Humano-Agente (el último bloque de código):
-- Cargamos la tabla Q que el agente entrenado generó.
Creamos una interfaz donde un jugador humano puede jugar partidas de Connect 4 directamente contra el agente entrenado.
El agente utiliza la tabla Q cargada para tomar sus decisiones de juego (siempre eligiendo la mejor opción conocida, sin exploración), mientras que el humano ingresa sus movimientos.
